In [1]:
# ============================================================
# 03_fine_tuning.ipynb
# Fine-tuning du modèle InceptionV3 sur les fleurs
# ============================================================

# ------------------------------------------------------------
# 1. Importations
# ------------------------------------------------------------
import os
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pickle


In [2]:
# ------------------------------------------------------------
# 2. Chemins et dossiers
# ------------------------------------------------------------
DATASET_PATH = "../data/flower_images"
MODEL_DIR = "../models"
os.makedirs(MODEL_DIR, exist_ok=True)

MODEL_BASELINE_PATH = os.path.join(MODEL_DIR, "inception_baseline.h5")

# Nom des fichiers avec timestamp pour versionner
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
MODEL_FINETUNED_PATH = os.path.join(MODEL_DIR, f"inception_finetuned_{timestamp}.h5")
HISTORY_FINETUNE_PATH = os.path.join(MODEL_DIR, f"history_finetune_{timestamp}.pkl")

print("Dataset :", DATASET_PATH)
print("Classes :", os.listdir(DATASET_PATH))


Dataset : ../data/flower_images
Classes : ['Lilly', 'Lotus', 'Orchid', 'Sunflower', 'Tulip']


In [3]:
# ------------------------------------------------------------
# 3. Paramètres
# ------------------------------------------------------------
IMG_SIZE = (299, 299)
BATCH_SIZE = 32
EPOCHS = 25
LR = 1e-5  # très faible pour fine-tuning


In [4]:
# ------------------------------------------------------------
# 4. Générateurs de données
# ------------------------------------------------------------
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1
)

train_gen = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

val_gen = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)


Found 4000 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.


In [5]:
# ------------------------------------------------------------
# 5. Charger le modèle baseline
# ------------------------------------------------------------
print("\nChargement du modèle baseline...")
baseline_model = load_model(MODEL_BASELINE_PATH)
baseline_model.summary()



Chargement du modèle baseline...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 299, 299, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 149, 149, 32)      │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 149, 149, 32)      │              96 │ conv2d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation (Activation)       │ (None, 149, 149, 32)      │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 147, 147, 32)      │           9,216 │ activation[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 147, 147, 32)      │              96 │ conv2d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_1 (Activation)     │ (None, 147, 147, 32)      │               0 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 147, 147, 64)      │          18,432 │ activation_1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 147, 147, 64)      │             192 │ conv2d_2[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_2 (Activation)     │ (None, 147, 147, 64)      │               0 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 73, 73, 64)        │               0 │ activation_2[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 73, 73, 80)        │           5,120 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_3         │ (None, 73, 73, 80)        │             240 │ conv2d_3[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_3 (Activation)     │ (None, 73, 73, 80)        │               0 │ batch_normalization_3[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 71, 71, 192)       │         138,240 │ activation_3[0][0]         │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 21,813,031 (83.21 MB)

 Trainable params: 10,245 (40.02 KB)

 Non-trainable params: 21,802,784 (83.17 MB)

 Optimizer params: 2 (12.00 B)

In [6]:
# ------------------------------------------------------------
# 6. Déverrouiller certaines couches (fine-tuning)
# ------------------------------------------------------------
print("\nDégel des couches profondes pour fine-tuning...\n")
fine_tune_at = 250  # laisser 250 premières couches gelées

for layer in baseline_model.layers[:fine_tune_at]:
    layer.trainable = False
for layer in baseline_model.layers[fine_tune_at:]:
    layer.trainable = True



Dégel des couches profondes pour fine-tuning...



In [7]:
# ------------------------------------------------------------
# 7. Compilation
# ------------------------------------------------------------
baseline_model.compile(
    optimizer=Adam(learning_rate=LR),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [8]:
# ------------------------------------------------------------
# 8. Callbacks pour entraîner efficacement
# ------------------------------------------------------------
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=3, 
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2, patience=2,
    verbose=1
)


In [9]:
# ------------------------------------------------------------
# 9. Entraînement du modèle fine-tuné
# ------------------------------------------------------------
history_finetune = baseline_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=[early_stop, reduce_lr]
)


C:\Users\pc\miniconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 636s 5s/step - accuracy: 0.7567 - loss: 0.7064 - val_accuracy: 0.8470 - val_loss: 0.4698 - learning_rate: 1.0000e-05
Epoch 2/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 592s 5s/step - accuracy: 0.8405 - loss: 0.4925 - val_accuracy: 0.8720 - val_loss: 0.3926 - learning_rate: 1.0000e-05
Epoch 3/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 591s 5s/step - accuracy: 0.8683 - loss: 0.4018 - val_accuracy: 0.8890 - val_loss: 0.3380 - learning_rate: 1.0000e-05
Epoch 4/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 595s 5s/step - accuracy: 0.8880 - loss: 0.3476 - val_accuracy: 0.9000 - val_loss: 0.3120 - learning_rate: 1.0000e-05
Epoch 5/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 595s 5s/step - accuracy: 0.9078 - loss: 0.2900 - val_accuracy: 0.9070 - val_loss: 0.2820 - learning_rate: 1.0000e-05
Epoch 6/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 624s 5s/step - accuracy: 0.9165 - loss: 0.2720 - val_accuracy: 0.9120 - val_loss: 0.2486 - learning_rate: 1.0000e-05
Epoch 7/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 591s 5s/step - acc

In [10]:
# ------------------------------------------------------------
# 10. Sauvegarde modèle et historique
# ------------------------------------------------------------
baseline_model.save(MODEL_FINETUNED_PATH)
with open(HISTORY_FINETUNE_PATH, "wb") as f:
    pickle.dump(history_finetune.history, f)

print("\n✔ Modèle fine-tuné sauvegardé :", MODEL_FINETUNED_PATH)
print("✔ Historique sauvegardé :", HISTORY_FINETUNE_PATH)



✔ Modèle fine-tuné sauvegardé : ../models\inception_finetuned_20251126-232904.h5
✔ Historique sauvegardé : ../models\history_finetune_20251126-232904.pkl
